In [1]:
#### install the required packages
# No module named 'natsort' : conda install natsort
# No module named 'polars' : conda install -c conda-forge polars
# No module named 'sqlalchemy' : conda install -c anaconda sqlalchemy
# No module named 'statsmodels' : conda install -c conda-forge statsmodels
# No module named 'sqlean' : pip install sqlean.py , pip install sqlite-utils
# No module named 'clustcrdist' : pip install clustcrdist
# No module named 'faiss' : conda install -c conda-forge faiss-cpu

import pandas as pd
from pathlib import Path
import os
import gzip

from raptcr.io.pipeline import ProcessingPipeline
from raptcr.io.mappers import RegexMapper
from raptcr.neighborhood import ConvergenceAnalysis, Fisher
from raptcr.hashing import TCRDistEmbedder
from clustcrdist.background import BackgroundModel

2025-03-11 16:46:25 - RepertoireReader - INFO - Logging initialized
2025-03-11 16:46:25 - faiss.loader - INFO - Loading faiss with AVX2 support.
2025-03-11 16:46:25 - faiss.loader - INFO - Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2025-03-11 16:46:25 - faiss.loader - INFO - Loading faiss.
2025-03-11 16:46:25 - faiss.loader - INFO - Successfully loaded faiss.
c:\Users\fried\miniconda3\envs\env\lib\site-packages\Bio\Application\__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [2]:
#check scipy version
import scipy
print(scipy.__version__)

1.15.2


In [3]:
import sqlean
print(dir(sqlean))


['Binary', 'Cache', 'Connection', 'Cursor', 'DataError', 'DatabaseError', 'Date', 'DateFromTicks', 'Error', 'IntegrityError', 'InterfaceError', 'InternalError', 'NotSupportedError', 'OperationalError', 'OptimizedUnicode', 'PARSE_COLNAMES', 'PARSE_DECLTYPES', 'PrepareProtocol', 'ProgrammingError', 'Row', 'SQLITE_ABORT', 'SQLITE_ALTER_TABLE', 'SQLITE_ANALYZE', 'SQLITE_ATTACH', 'SQLITE_AUTH', 'SQLITE_BUSY', 'SQLITE_CANTOPEN', 'SQLITE_CONSTRAINT', 'SQLITE_CORRUPT', 'SQLITE_CREATE_INDEX', 'SQLITE_CREATE_TABLE', 'SQLITE_CREATE_TEMP_INDEX', 'SQLITE_CREATE_TEMP_TABLE', 'SQLITE_CREATE_TEMP_TRIGGER', 'SQLITE_CREATE_TEMP_VIEW', 'SQLITE_CREATE_TRIGGER', 'SQLITE_CREATE_VIEW', 'SQLITE_CREATE_VTABLE', 'SQLITE_DELETE', 'SQLITE_DENY', 'SQLITE_DETACH', 'SQLITE_DONE', 'SQLITE_DROP_INDEX', 'SQLITE_DROP_TABLE', 'SQLITE_DROP_TEMP_INDEX', 'SQLITE_DROP_TEMP_TABLE', 'SQLITE_DROP_TEMP_TRIGGER', 'SQLITE_DROP_TEMP_VIEW', 'SQLITE_DROP_TRIGGER', 'SQLITE_DROP_VIEW', 'SQLITE_DROP_VTABLE', 'SQLITE_EMPTY', 'SQLITE_ERRO

In [4]:
path = os.getcwd()
data_path = path + '\\data\\HLA_emerson_2017'
# Extract patient id's from the file names where the patient id is the first part of the file name
# and is separated by an underscore
patient_files = os.listdir(data_path)
patient_ids = [file.split('.')[0] for file in patient_files]
patient_ids[:5] 

['P00001_clonotypes',
 'P00002_clonotypes',
 'P00003_clonotypes',
 'P00004_clonotypes',
 'P00005_clonotypes']

In [10]:
#pipe = ProcessingPipeline(
#    patient_id_mapping=patient_ids,
#    repertoire_id_mapping=patient_ids
#)

In [34]:
#data = pipe.process_dataset(data_path)

In [5]:
#read all files in the directory, make a dataframe with all the data of the patients concatenated
#read only the first 5 patient files for now
tsv_gz_files = [f for f in patient_files if f.endswith('.tsv.gz')][:5]
data = pd.DataFrame()
for file in tsv_gz_files:
    file_path = os.path.join(data_path, file)
    with gzip.open(file_path, 'rb') as f:
        df = pd.read_csv(f, sep='\t')
        data = pd.concat([data, df])



In [6]:
data

,junction,v_call,junction_aa,j_call,duplicate_count,repertoire_id
0,gtgccaccacgggtactagcgggggcccaagccagagtacgcagta...,TRBV10-3,CATTGTSGGPSQSTQYF,TRBJ2-3,44446,P00001_clonotypes
1,gtgccagcagcttacgagtcgggggctatggctacaccttc,TRBV7-9,CASSLRVGGYGYTF,TRBJ1-2,44049,P00001_clonotypes
2,gtgccagcagacgcctcggagggttgaacactgaagctttcttt,TRBV25-1,CASRRLGGLNTEAFF,TRBJ1-1,24891,P00001_clonotypes
3,gcgccagcagcttgagaggctctggaaacaccatatatttt,TRBV5-1,CASSLRGSGNTIYF,TRBJ1-3,14303,P00001_clonotypes
4,gtgccagcagacaggacgggagcactgaagctttcttt,TRBV6-1,CASRQDGSTEAFF,TRBJ1-1,12955,P00001_clonotypes
...,...,...,...,...,...,...
76063,gtgccagcagcggggcagcgggagtcttcttc,TRBV9,CASSGAAGVFF,TRBJ2-4,2,P00005_clonotypes
76064,gcgccagcagcttatccggggcggcctacgagcagtacttc,TRBV5-1,CASSLSGAAYEQYF,TRBJ2-5,2,P00005_clonotypes
76065,gtgccagcagtttagtgggacagggggtctacgagcagtacttc,TRBV27,CASSLVGQGVYEQYF,TRBJ2-5,2,P00005_clonotypes
76066,gtgccagcagcggggccgggacaggggttagtggctacaccttc,TRBV9,CASSGAGTGVSGYTF,TRBJ1-2,2,P00005_clonotypes


In [7]:
data["chain"] = data["v_call"].str[:3]
data = data.query("chain == 'TRB'").reset_index(drop=True)

In [8]:
data

,junction,v_call,junction_aa,j_call,duplicate_count,repertoire_id,chain
0,gtgccaccacgggtactagcgggggcccaagccagagtacgcagta...,TRBV10-3,CATTGTSGGPSQSTQYF,TRBJ2-3,44446,P00001_clonotypes,TRB
1,gtgccagcagcttacgagtcgggggctatggctacaccttc,TRBV7-9,CASSLRVGGYGYTF,TRBJ1-2,44049,P00001_clonotypes,TRB
2,gtgccagcagacgcctcggagggttgaacactgaagctttcttt,TRBV25-1,CASRRLGGLNTEAFF,TRBJ1-1,24891,P00001_clonotypes,TRB
3,gcgccagcagcttgagaggctctggaaacaccatatatttt,TRBV5-1,CASSLRGSGNTIYF,TRBJ1-3,14303,P00001_clonotypes,TRB
4,gtgccagcagacaggacgggagcactgaagctttcttt,TRBV6-1,CASRQDGSTEAFF,TRBJ1-1,12955,P00001_clonotypes,TRB
...,...,...,...,...,...,...,...
501750,gtgccagcagcggggcagcgggagtcttcttc,TRBV9,CASSGAAGVFF,TRBJ2-4,2,P00005_clonotypes,TRB
501751,gcgccagcagcttatccggggcggcctacgagcagtacttc,TRBV5-1,CASSLSGAAYEQYF,TRBJ2-5,2,P00005_clonotypes,TRB
501752,gtgccagcagtttagtgggacagggggtctacgagcagtacttc,TRBV27,CASSLVGQGVYEQYF,TRBJ2-5,2,P00005_clonotypes,TRB
501753,gtgccagcagcggggccgggacaggggttagtggctacaccttc,TRBV9,CASSGAGTGVSGYTF,TRBJ1-2,2,P00005_clonotypes,TRB


In [57]:
#data.rename(columns={"v_call": "v_gene", "j_call" : "j_gene"}, inplace=True)
#data["v_call"] = data["v_gene"]+"*01"
#data["j_call"] = data["j_gene"]+"*01"

In [58]:
#data.head()

In [9]:
def read_patient_files(patients_path, number_of_patients=666, training=True):
    files_path = path + patients_path #'\\data\\HLA_emerson_2017'

    all_files = os.listdir(files_path)

    tsv_gz_files = [f for f in all_files if f.endswith('.tsv.gz')]

    if training == True:
        tsv_gz_files = tsv_gz_files[:number_of_patients] #tsv_gz_files[:400]

    elif training == False:
        tsv_gz_files = tsv_gz_files[number_of_patients:]

    patient_names = [f.split('.')[0] for f in tsv_gz_files]

    return tsv_gz_files, patient_names

def match_hla(patients_path, hla_path):
    tsv_gz_files, all_patient_names = read_patient_files(patients_path)

    record =  {}  # Temporary dictionary to hold one record
    file_path = path + hla_path #'\\data\\A_02_01_features.txt'

    # Read and parse the file
    with open(file_path, "r") as file:
        for line in file:
            line = line.split(" ")
            if line[1] == 'HLA-A*02:01':
                key = line[2]
                for i in range(5, len(line)):
                    value = line[i]
                    if value == '\n':
                        continue
                    ## value is the index to the patient file
                    patient = all_patient_names[int(value)]
                    if key in ['num_positives:']:
                        record[patient] = True
                    elif key in ['num_negatives:']:
                        record[patient] = False   


    for patient in all_patient_names:
        if patient not in record.keys():
            record[patient] = None

    patient_df = pd.DataFrame(list(record.items()), columns=['repertoire_id', 'has_HLA_A02_01'])
    patient_df.sort_values(by=['repertoire_id'], inplace=True, ignore_index=True)
    return patient_df

In [10]:
patient_df = match_hla('\\data\\HLA_emerson_2017', '\\data\\A_02_01_features.txt')
patient_df

,repertoire_id,has_HLA_A02_01
0,P00001_clonotypes,False
1,P00002_clonotypes,True
2,P00003_clonotypes,True
3,P00004_clonotypes,True
4,P00005_clonotypes,False
...,...,...
661,P00663_clonotypes,True
662,P00665_clonotypes,True
663,P00666_clonotypes,False
664,P00667_clonotypes,False


In [11]:
#select the patients that are in the data dataframe (column 'repertoire_id')
patient_df = patient_df[patient_df['repertoire_id'].isin(data['repertoire_id'])]
patient_df

,repertoire_id,has_HLA_A02_01
0,P00001_clonotypes,False
1,P00002_clonotypes,True
2,P00003_clonotypes,True
3,P00004_clonotypes,True
4,P00005_clonotypes,False


In [12]:
#select only the patients that have label true in patient_df
positives = patient_df[patient_df['has_HLA_A02_01'] == True]
#make a list of the patients that are positives where the list containts the repertoire_id
positives = list(positives['repertoire_id'])
positives

['P00002_clonotypes', 'P00003_clonotypes', 'P00004_clonotypes']

In [13]:
tcr_embedder = TCRDistEmbedder(full_tcr=False).fit() # full tcr not needed if grouping by v_call

c:\Users\fried\miniconda3\envs\env\lib\site-packages\raptcr\constants\preprocessing.py:113: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  L = len(cdrs[0])


In [14]:
fisher = Fisher(
    group_column="repertoire_id", # compare values in the repertoire_id column
    positive_groups=positives # compare non-background (positive group) to background (negative group)
)

In [15]:
# pass the embedder and the fisher method to the ConvergenceAnalysis object:

cva = ConvergenceAnalysis(
    tcr_embedder=tcr_embedder,
    convergence_metric=fisher,
    verbose=True
)

In [16]:
cva_res = cva.batched_fit_transform(
    data
    )

Processing V gene TRBV10-1 (0/43)
Finished searching neighbors (0.77 seconds).
Finished calculating convergence (0.44 seconds).
Processing V gene TRBV10-2 (1/43)
Finished searching neighbors (0.94 seconds).
Finished calculating convergence (0.29 seconds).
Processing V gene TRBV10-3 (2/43)
Finished searching neighbors (3.70 seconds).
Finished calculating convergence (1.05 seconds).
Processing V gene TRBV11-1 (3/43)
Finished searching neighbors (0.53 seconds).
Finished calculating convergence (0.08 seconds).
Processing V gene TRBV11-2 (4/43)
Finished searching neighbors (4.37 seconds).
Finished calculating convergence (0.96 seconds).
Processing V gene TRBV11-3 (5/43)
Finished searching neighbors (1.40 seconds).
Finished calculating convergence (0.22 seconds).
Processing V gene TRBV12-5 (6/43)
Finished searching neighbors (0.72 seconds).
Finished calculating convergence (0.27 seconds).
Processing V gene TRBV13 (7/43)
Finished searching neighbors (1.46 seconds).
Finished calculating conver

c:\Users\fried\miniconda3\envs\env\lib\site-packages\raptcr\neighborhood.py:407: UserWarning: 72.90% of matches were not found within the index. Consider increasing the nprobe.
  warnings.warn(f"{frac_nan:.2%} of matches were not found within the index. Consider increasing the nprobe.")


Finished searching neighbors (5.12 seconds).
Finished calculating convergence (2.07 seconds).
Processing V gene TRBV19 (12/43)
Finished searching neighbors (13.27 seconds).
Finished calculating convergence (2.33 seconds).
Processing V gene TRBV2 (13/43)
Finished searching neighbors (9.50 seconds).
Finished calculating convergence (1.36 seconds).
Processing V gene TRBV20-1 (14/43)
Finished searching neighbors (0.68 seconds).
Finished calculating convergence (0.87 seconds).
Processing V gene TRBV25-1 (15/43)
Finished searching neighbors (2.00 seconds).
Finished calculating convergence (0.20 seconds).
Processing V gene TRBV27 (16/43)
Finished searching neighbors (9.85 seconds).
Finished calculating convergence (1.75 seconds).
Processing V gene TRBV28 (17/43)
Finished searching neighbors (9.90 seconds).
Finished calculating convergence (2.12 seconds).
Processing V gene TRBV29-1 (18/43)
Finished searching neighbors (6.16 seconds).
Finished calculating convergence (1.46 seconds).
Processing 

c:\Users\fried\miniconda3\envs\env\lib\site-packages\raptcr\neighborhood.py:407: UserWarning: 99.61% of matches were not found within the index. Consider increasing the nprobe.
  warnings.warn(f"{frac_nan:.2%} of matches were not found within the index. Consider increasing the nprobe.")


Finished searching neighbors (7.19 seconds).
Finished calculating convergence (1.70 seconds).
Processing V gene TRBV4-1 (21/43)
Finished searching neighbors (4.53 seconds).
Finished calculating convergence (0.48 seconds).
Processing V gene TRBV4-2 (22/43)
Finished searching neighbors (3.06 seconds).
Finished calculating convergence (0.28 seconds).
Processing V gene TRBV4-3 (23/43)
Finished searching neighbors (2.95 seconds).
Finished calculating convergence (0.52 seconds).
Processing V gene TRBV5-1 (24/43)
Finished searching neighbors (16.84 seconds).
Finished calculating convergence (3.01 seconds).
Processing V gene TRBV5-4 (25/43)
Finished searching neighbors (5.19 seconds).
Finished calculating convergence (0.61 seconds).
Processing V gene TRBV5-5 (26/43)
Finished searching neighbors (1.34 seconds).
Finished calculating convergence (0.11 seconds).
Processing V gene TRBV5-6 (27/43)
Finished searching neighbors (4.41 seconds).
Finished calculating convergence (0.59 seconds).
Processin

c:\Users\fried\miniconda3\envs\env\lib\site-packages\raptcr\neighborhood.py:407: UserWarning: 94.24% of matches were not found within the index. Consider increasing the nprobe.
  warnings.warn(f"{frac_nan:.2%} of matches were not found within the index. Consider increasing the nprobe.")
c:\Users\fried\miniconda3\envs\env\lib\site-packages\raptcr\neighborhood.py:407: UserWarning: 93.41% of matches were not found within the index. Consider increasing the nprobe.
  warnings.warn(f"{frac_nan:.2%} of matches were not found within the index. Consider increasing the nprobe.")


Finished searching neighbors (8.01 seconds).
Finished calculating convergence (1.27 seconds).
Processing V gene TRBV7-3 (36/43)
Finished searching neighbors (2.57 seconds).
Finished calculating convergence (0.33 seconds).
Processing V gene TRBV7-4 (37/43)
Finished searching neighbors (0.01 seconds).
Finished calculating convergence (0.01 seconds).
Processing V gene TRBV7-6 (38/43)


c:\Users\fried\miniconda3\envs\env\lib\site-packages\raptcr\neighborhood.py:407: UserWarning: 94.43% of matches were not found within the index. Consider increasing the nprobe.
  warnings.warn(f"{frac_nan:.2%} of matches were not found within the index. Consider increasing the nprobe.")


Finished searching neighbors (2.33 seconds).
Finished calculating convergence (0.20 seconds).
Processing V gene TRBV7-7 (39/43)
Finished searching neighbors (0.58 seconds).
Finished calculating convergence (0.06 seconds).
Processing V gene TRBV7-8 (40/43)
Finished searching neighbors (3.46 seconds).
Finished calculating convergence (0.26 seconds).
Processing V gene TRBV7-9 (41/43)
Finished searching neighbors (12.05 seconds).
Finished calculating convergence (1.83 seconds).
Processing V gene TRBV9 (42/43)
Finished searching neighbors (10.53 seconds).
Finished calculating convergence (2.68 seconds).


In [59]:
#get types of objects in data
data.dtypes

junction           object
v_call             object
junction_aa        object
j_call             object
duplicate_count     int64
repertoire_id      object
chain              object
dtype: object

In [17]:
cva_res

,junction,v_call,junction_aa,j_call,duplicate_count,repertoire_id,chain,match_true,match_false,background_true,background_false,statistic,pvalue,convergence
0,gcgccagcagtgacagggacaccggggagctgtttttt,TRBV10-1*01,CASSDRDTGELFF,TRBJ2-2,2016,P00001_clonotypes,TRB,3,2,1329,718,1.079819,1.000000,-0.303323
1,gcgccagcagtacatcccacgagcagtacttc,TRBV10-1*01,CASSTSHEQYF,TRBJ2-7,1686,P00001_clonotypes,TRB,1,2,1327,722,0.000000,0.352539,-1.878098
2,gcgccagcagtgagtcgtacggcagggggtcgtcggacactgaagc...,TRBV10-1*01,CASSESYGRGSSDTEAFF,TRBJ1-1,555,P00001_clonotypes,TRB,1,2,1328,721,0.000000,0.352051,-1.881184
3,gcgccagcagctacaggggtgaagctttcttt,TRBV10-1*01,CASSYRGEAFF,TRBJ1-1,473,P00001_clonotypes,TRB,1,2,1330,719,0.000000,0.351074,-1.887363
4,gcgccagcagtgcgcccaactatggctacaccttc,TRBV10-1*01,CASSAPNYGYTF,TRBJ1-2,378,P00001_clonotypes,TRB,1,2,1336,713,0.000000,0.348145,-1.905946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501750,gtgccagcagcggggacaaaaactatggctacaccttc,TRBV9*01,CASSGDKNYGYTF,TRBJ1-2,2,P00005_clonotypes,TRB,1,2,1238,811,0.000000,0.395996,-1.610237
501751,gtgccagcagacaggaagggggtgaagctttcttt,TRBV9*01,CASRQEGGEAFF,TRBJ1-1,2,P00005_clonotypes,TRB,1,2,1166,883,0.000000,0.431152,-1.401082
501752,gtgccagcagcggggcgacagggggatacgctagcacagatacgca...,TRBV9*01,CASSGATGGYASTDTQYF,TRBJ2-3,2,P00005_clonotypes,TRB,1,2,1297,752,0.000000,0.367188,-1.786374
501753,gtgccagcagcggggcagcgggagtcttcttc,TRBV9*01,CASSGAAGVFF,TRBJ2-4,2,P00005_clonotypes,TRB,1,2,1138,911,0.000000,0.444824,-1.320978
